In [8]:
import cantera as ct
import numpy as np

In [9]:
# Use air for stream a.
gas_a = ct.Solution('air.xml')
gas_a.TPX = 300.0, ct.one_atm, 'O2:0.21, N2:0.78, AR:0.01'
rho_a = gas_a.density

In [10]:
# Use GRI-Mech 3.0 for stream b (methane) and for the mixer. If it is desired
# to have a pure mixer, with no chemistry, use instead a reaction mechanism
# for gas_b that has no reactions.
gas_b = ct.Solution('gri30.xml')
gas_b.TPX = 300.0, ct.one_atm, 'CH4:1'
rho_b = gas_b.density

In [11]:
# Create reservoirs for the two inlet streams and for the outlet stream.  The
# upsteam reservoirs could be replaced by reactors, which might themselves be
# connected to reactors further upstream. The outlet reservoir could be
# replaced with a reactor with no outlet, if it is desired to integrate the
# composition leaving the mixer in time, or by an arbitrary network of
# downstream reactors.
res_a = ct.Reservoir(gas_a)
res_b = ct.Reservoir(gas_b)
downstream = ct.Reservoir(gas_b)

In [12]:
# Create a reactor for the mixer. A reactor is required instead of a
# reservoir, since the state will change with time if the inlet mass flow
# rates change or if there is chemistry occurring.
gas_b.TPX = 300.0, ct.one_atm, 'O2:0.21, N2:0.78, AR:0.01'
mixer = ct.IdealGasReactor(gas_b)

In [13]:
# create two mass flow controllers connecting the upstream reservoirs to the
# mixer, and set their mass flow rates to values corresponding to
# stoichiometric combustion.
mfc1 = ct.MassFlowController(res_a, mixer, mdot=rho_a*2.5/0.21)
mfc2 = ct.MassFlowController(res_b, mixer, mdot=rho_b*1.0)

In [14]:
# connect the mixer to the downstream reservoir with a valve.
outlet = ct.Valve(mixer, downstream, K=10.0)

In [15]:
sim = ct.ReactorNet([mixer])

In [16]:
# Since the mixer is a reactor, we need to integrate in time to reach steady
# state
sim.advance_to_steady_state()

In [17]:
# view the state of the gas in the mixer
print(mixer.thermo.report())


  gri30:

       temperature             300  K
          pressure          101326  Pa
           density         1.13613  kg/m^3
  mean mol. weight          27.968  amu

                          1 kg            1 kmol
                       -----------      ------------
          enthalpy      -2.047e+05       -5.725e+06     J
   internal energy     -2.9388e+05       -8.219e+06     J
           entropy          7158.9        2.002e+05     J/K
    Gibbs function     -2.3524e+06       -6.579e+07     J
 heat capacity c_p          1057.6        2.958e+04     J/K
 heat capacity c_v          760.27        2.126e+04     J/K

                           X                 Y          Chem. Pot. / RT
                     -------------     ------------     ------------
                O2       0.193727         0.221647          -26.315
               CH4      0.0774908        0.0444495         -54.8804
                N2       0.719557         0.720727         -23.3622
                AR     0.0